# Surface Data Preparation

Extract session-z-scored, rep-averaged surface betas for ALL 1000 shared images,
all 4 ROIs (V1, V2, V4, IT), all 8 subjects.

**Output:** Per-region xarray DataArrays saved as netCDF.
- `Allen2022_surface.V1.nc`, `Allen2022_surface.V2.nc`, etc.
- Contains 1000 images (filtered to 515 at benchmark load time)
- `min_reps_across_subjects` coordinate enables filtering

**Processing chain:** raw MGH betas -> session z-score -> average reps

**Estimated runtime:** ~35 min (568 MGH file loads from external drive)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import nibabel as nib
import scipy.io
import time
from pathlib import Path
from tqdm.auto import tqdm

NSD_ROOT = Path('/Volumes/Hagibis/nsd')
FSAVG_LABELS = NSD_ROOT / 'fsaverage_labels'
OUTPUT_DIR = NSD_ROOT / 'assemblies'
OUTPUT_DIR.mkdir(exist_ok=True)

# --- Subject Configuration ---
# Use all 8 subjects (515 shared images with 3 reps):
SUBJECT_LIST = [1, 2, 3, 4, 5, 6, 7, 8]
# Or use only 4 complete subjects (40 sessions each, ~1000 images):
# SUBJECT_LIST = [1, 2, 5, 7]

N_SUBJECTS = len(SUBJECT_LIST)
ALL_SESSIONS = {1: 40, 2: 40, 3: 32, 4: 30, 5: 40, 6: 32, 7: 40, 8: 30}
SESSIONS_PER_SUBJECT = {s: ALL_SESSIONS[s] for s in SUBJECT_LIST}

TRIALS_PER_SESSION = 750
N_FSAVG_VERTICES = 163842

## 1. ROI Masks and Constants

In [ ]:
# Load ROI labels
lh_kastner = nib.load(str(FSAVG_LABELS / 'lh.Kastner2015.mgz')).get_fdata().flatten()
rh_kastner = nib.load(str(FSAVG_LABELS / 'rh.Kastner2015.mgz')).get_fdata().flatten()
lh_streams = nib.load(str(FSAVG_LABELS / 'lh.streams.mgz')).get_fdata().flatten()
rh_streams = nib.load(str(FSAVG_LABELS / 'rh.streams.mgz')).get_fdata().flatten()

REGION_TO_KASTNER_LABELS = {
    'V1': [1, 2],   # V1v, V1d
    'V2': [3, 4],   # V2v, V2d
    'V4': [7],       # hV4
}

# IT: NSD "streams" ventral parcellation (label 5), consistent with Algonauts 2023.
# Replaces the earlier custom Glasser HCP_MMP1 9-parcel definition.
STREAMS_VENTRAL_LABEL = 5

REGIONS = ['V1', 'V2', 'V4', 'IT']

# Build masks: region -> {'lh': bool(163842,), 'rh': bool(163842,)}
roi_masks = {}
for region, labels in REGION_TO_KASTNER_LABELS.items():
    roi_masks[region] = {
        'lh': np.isin(lh_kastner, labels),
        'rh': np.isin(rh_kastner, labels),
    }
roi_masks['IT'] = {
    'lh': lh_streams == STREAMS_VENTRAL_LABEL,
    'rh': rh_streams == STREAMS_VENTRAL_LABEL,
}

for region in REGIONS:
    n_lh = roi_masks[region]['lh'].sum()
    n_rh = roi_masks[region]['rh'].sum()
    print(f'{region}: LH={n_lh}, RH={n_rh}, total={n_lh + n_rh}')

## 2. Trial Mapping for All 1000 Shared Images

In [ ]:
# Load experiment design
stim_info = pd.read_csv(NSD_ROOT / 'metadata' / 'nsd_stim_info_merged.csv', index_col=0)
expdesign = scipy.io.loadmat(str(NSD_ROOT / 'metadata' / 'nsd_expdesign.mat'))
masterordering = expdesign['masterordering'].flatten()
subjectim = expdesign['subjectim']
sharedix = expdesign['sharedix'].flatten()  # 1-indexed nsdIds


def get_shared_trial_info(subj_idx: int) -> pd.DataFrame:
    """Find trial indices for shared images for a given subject.
    
    Returns DataFrame with columns:
        nsd_id: 0-indexed NSD image ID
        shared_pos: 0-indexed position in the 1000 shared images
        rep: repetition number (0, 1, 2)
        session: 1-indexed session number
        trial_in_session: 0-indexed position within session
    """
    subj = subj_idx + 1
    n_sessions = SESSIONS_PER_SUBJECT[subj]
    n_total_trials = n_sessions * TRIALS_PER_SESSION
    
    subj_nsdids = subjectim[subj_idx]
    nsdid_to_imgidx = {int(nsd_id): img_idx + 1 
                       for img_idx, nsd_id in enumerate(subj_nsdids)}
    
    # Map shared nsdIds to image indices for this subject
    shared_imgidxs = {}
    for shared_pos, nsd_id in enumerate(sharedix):
        nsd_id_int = int(nsd_id)
        if nsd_id_int in nsdid_to_imgidx:
            shared_imgidxs[nsdid_to_imgidx[nsd_id_int]] = (shared_pos, nsd_id_int - 1)
    
    records = []
    rep_counter = {}
    for trial_idx in range(n_total_trials):
        img_idx = masterordering[trial_idx]
        if img_idx in shared_imgidxs:
            shared_pos, nsd_id_0 = shared_imgidxs[img_idx]
            rep = rep_counter.get(img_idx, 0)
            rep_counter[img_idx] = rep + 1
            session = trial_idx // TRIALS_PER_SESSION + 1
            trial_in_session = trial_idx % TRIALS_PER_SESSION
            records.append({
                'nsd_id': nsd_id_0,
                'shared_pos': shared_pos,
                'rep': rep,
                'session': session,
                'trial_in_session': trial_in_session,
            })
    
    return pd.DataFrame(records)


# Build trial info for all subjects in SUBJECT_LIST
all_trial_info = {}
for subj in SUBJECT_LIST:
    subj_idx = subj - 1
    df = get_shared_trial_info(subj_idx)
    all_trial_info[subj] = df
    n_images = df['shared_pos'].nunique()
    n_trials = len(df)
    rep_counts = df.groupby('shared_pos')['rep'].max().value_counts().sort_index()
    print(f'subj{subj:02d}: {n_trials} trials, {n_images} unique images, '
          f'rep distribution: {dict(rep_counts)}')

In [ ]:
# Compute min_reps_across_subjects for each of the 1000 shared images
reps_per_image = np.zeros((N_SUBJECTS, 1000), dtype=int)
for i, subj in enumerate(SUBJECT_LIST):
    df = all_trial_info[subj]
    for shared_pos, group in df.groupby('shared_pos'):
        reps_per_image[i, shared_pos] = len(group)

min_reps = reps_per_image.min(axis=0)
shared_nsdids = sharedix - 1  # 0-indexed

print(f'Subjects: {SUBJECT_LIST}')
print(f'min_reps distribution:')
for r in range(4):
    print(f'  {r}: {(min_reps == r).sum()} images')
print(f'\nImages with 3 complete reps: {(min_reps >= 3).sum()}')

## 3. Core Extraction

For each subject:
1. Walk through sessions, load LH and RH surface betas
2. Extract ROI vertices
3. Z-score within session (750 trials per vertex)
4. Collect shared-image trial betas
5. Average across repetitions

In [ ]:
def extract_surface_betas_for_subject(subj: int, trial_info: pd.DataFrame):
    """Extract shared-image betas for all regions from surface data.
    
    Returns:
        region_betas: dict[region -> (1000, n_vertices) averaged betas]
        region_per_rep: dict[region -> (1000, max_reps, n_vertices) per-rep betas]
    """
    n_sessions = SESSIONS_PER_SUBJECT[subj]
    
    # Pre-allocate per-region storage
    # For each region, collect (shared_pos, rep, beta_vector)
    region_trials = {region: [] for region in REGIONS}
    
    for session in tqdm(range(1, n_sessions + 1), desc=f'subj{subj:02d}', leave=False):
        session_trials = trial_info[trial_info['session'] == session]
        if len(session_trials) == 0:
            continue
        
        # Load LH betas, extract ROI vertices, z-score, then free memory
        lh_path = NSD_ROOT / f'subj{subj:02d}' / 'betas' / f'lh.betas_session{session:02d}.mgh'
        lh_betas = nib.load(str(lh_path)).get_fdata().squeeze()  # (163842, 750)
        
        rh_path = NSD_ROOT / f'subj{subj:02d}' / 'betas' / f'rh.betas_session{session:02d}.mgh'
        rh_betas = nib.load(str(rh_path)).get_fdata().squeeze()  # (163842, 750)
        
        for region in REGIONS:
            lh_mask = roi_masks[region]['lh']
            rh_mask = roi_masks[region]['rh']
            
            # Extract ROI vertices: (n_trials, n_lh + n_rh)
            roi_betas = np.concatenate([
                lh_betas[lh_mask].T,   # (750, n_lh)
                rh_betas[rh_mask].T,   # (750, n_rh)
            ], axis=1)                 # (750, n_total)
            
            # Stage 1 of 2: Session z-score (Allen et al. 2022, Extended Data Fig. 8).
            # Normalize each vertex to mean=0, std=1 within each 750-trial session.
            # Removes within-session non-stationarities and equalizes units across
            # vertices. This is the only normalization the NSD paper prescribes.
            # Stage 2 (global z-score per subject) is applied later in NB03.
            mean = roi_betas.mean(axis=0, keepdims=True)
            std = roi_betas.std(axis=0, keepdims=True)
            std[std == 0] = 1.0
            roi_betas = (roi_betas - mean) / std
            
            # Collect shared-image trials
            for _, row in session_trials.iterrows():
                region_trials[region].append((
                    row['shared_pos'],
                    row['rep'],
                    roi_betas[row['trial_in_session']].copy(),
                ))
        
        del lh_betas, rh_betas
    
    # Organize into arrays
    region_averaged = {}
    region_per_rep = {}
    
    for region in REGIONS:
        n_verts = roi_masks[region]['lh'].sum() + roi_masks[region]['rh'].sum()
        
        # Build (1000, max_reps, n_verts) array
        max_reps = max(r for _, r, _ in region_trials[region]) + 1 if region_trials[region] else 0
        per_rep = np.full((1000, max(max_reps, 3), n_verts), np.nan, dtype=np.float32)
        
        for shared_pos, rep, beta_vec in region_trials[region]:
            per_rep[shared_pos, rep] = beta_vec
        
        # Average across available reps (nanmean handles missing reps)
        averaged = np.nanmean(per_rep, axis=1)  # (1000, n_verts)
        
        region_averaged[region] = averaged
        region_per_rep[region] = per_rep
    
    return region_averaged, region_per_rep

In [ ]:
# Extract betas for all subjects
# This is the main compute loop: ~35 min for 568 MGH file loads

all_averaged = {region: [] for region in REGIONS}  # region -> list of (1000, n_verts) per subject
all_per_rep = {region: [] for region in REGIONS}   # region -> list of (1000, 3, n_verts) per subject

t0 = time.time()

for subj in SUBJECT_LIST:
    print(f'\nProcessing subject {subj} ({SUBJECT_LIST.index(subj)+1}/{N_SUBJECTS})...')
    trial_info = all_trial_info[subj]
    averaged, per_rep = extract_surface_betas_for_subject(subj, trial_info)
    
    for region in REGIONS:
        all_averaged[region].append(averaged[region])
        all_per_rep[region].append(per_rep[region])
    
    elapsed = time.time() - t0
    print(f'  Done. Elapsed: {elapsed/60:.1f} min')

total_time = time.time() - t0
print(f'\nTotal extraction time: {total_time/60:.1f} min')

## 4. Compute Surface Noise Ceiling per Vertex

In [ ]:
def ncsnr_to_nc(ncsnr: np.ndarray, k: int = 3) -> np.ndarray:
    """Convert ncsnr to noise ceiling percentage."""
    return 100.0 * ncsnr**2 / (ncsnr**2 + 1.0 / k)


# Build per-vertex NC for all subjects, all regions
# Since ROI masks are shared across subjects (fsaverage), the vertex indices
# are the same for all subjects. But ncsnr differs per subject.

nc_per_subject_region = {}  # (subj, region) -> nc array

for subj in SUBJECT_LIST:
    lh_ncsnr = nib.load(str(NSD_ROOT / f'subj{subj:02d}' / 'betas' / 'lh.ncsnr.mgh')).get_fdata().flatten()
    rh_ncsnr = nib.load(str(NSD_ROOT / f'subj{subj:02d}' / 'betas' / 'rh.ncsnr.mgh')).get_fdata().flatten()
    
    for region in REGIONS:
        lh_mask = roi_masks[region]['lh']
        rh_mask = roi_masks[region]['rh']
        ncsnr_roi = np.concatenate([lh_ncsnr[lh_mask], rh_ncsnr[rh_mask]])
        nc_roi = ncsnr_to_nc(ncsnr_roi, k=3)
        nc_per_subject_region[(subj, region)] = nc_roi
        
        if subj == SUBJECT_LIST[0]:
            print(f'subj{subj:02d} {region}: {len(nc_roi)} vertices, '
                  f'median NC={np.nanmedian(nc_roi):.1f}%')

print('\nNC computed for all subjects and regions.')

## 5. Build Per-Region xarray DataArrays

In [ ]:
def build_region_assembly(region: str) -> xr.DataArray:
    """Build a DataArray for one region with all 1000 shared images, all subjects.
    
    Dimensions: (presentation=1000, neuroid=n_verts_per_subj * N_SUBJECTS)
    """
    lh_mask = roi_masks[region]['lh']
    rh_mask = roi_masks[region]['rh']
    n_lh = lh_mask.sum()
    n_rh = rh_mask.sum()
    n_verts = n_lh + n_rh
    lh_indices = np.where(lh_mask)[0]
    rh_indices = np.where(rh_mask)[0]
    
    # Stack subjects along neuroid dim: (1000, n_verts * N_SUBJECTS)
    data_list = all_averaged[region]  # list of (1000, n_verts) per subject
    data = np.concatenate(data_list, axis=1)  # (1000, n_verts * N_SUBJECTS)
    
    # Build neuroid coordinates
    neuroid_ids = []
    subjects = []
    hemispheres = []
    vertex_indices = []
    regions_coord = []
    nc_values = []
    
    for subj in SUBJECT_LIST:
        nc = nc_per_subject_region[(subj, region)]
        v_idx = 0
        for hemi, indices in [('lh', lh_indices), ('rh', rh_indices)]:
            for fsavg_idx in indices:
                neuroid_ids.append(f'subj{subj:02d}_{hemi}_{region}_v{v_idx:05d}')
                subjects.append(f'subj{subj:02d}')
                hemispheres.append(hemi)
                vertex_indices.append(int(fsavg_idx))
                regions_coord.append(region)
                nc_values.append(float(nc[v_idx]))
                v_idx += 1
    
    # Build presentation coordinates
    stimulus_ids = [f'nsd_{int(nid):05d}' for nid in shared_nsdids]
    
    da = xr.DataArray(
        data.astype(np.float32),
        dims=['presentation', 'neuroid'],
        coords={
            # Presentation coords
            'stimulus_id': ('presentation', stimulus_ids),
            'nsd_id': ('presentation', shared_nsdids.astype(int)),
            'min_reps_across_subjects': ('presentation', min_reps),
            # Neuroid coords
            'neuroid_id': ('neuroid', neuroid_ids),
            'subject': ('neuroid', subjects),
            'hemisphere': ('neuroid', hemispheres),
            'vertex_index': ('neuroid', vertex_indices),
            'region': ('neuroid', regions_coord),
            'nc_testset': ('neuroid', nc_values),
        },
    )
    
    return da


# Build and save per-region assemblies
for region in REGIONS:
    print(f'\nBuilding {region} assembly...')
    da = build_region_assembly(region)
    print(f'  Shape: {da.shape}')
    print(f'  Size: {da.nbytes / 1e6:.1f} MB')
    
    # Validate
    assert da.shape[0] == 1000
    assert len(np.unique(da.coords['subject'].values)) == N_SUBJECTS
    n_nan = np.isnan(da.values).sum()
    n_complete = (da.coords['min_reps_across_subjects'].values >= 3).sum()
    print(f'  NaN values: {n_nan} ({100*n_nan/da.values.size:.1f}%)')
    print(f'  Complete images (min_reps>=3): {n_complete}')
    
    # Save
    out_path = OUTPUT_DIR / f'Allen2022_surface.{region}.nc'
    da.to_netcdf(str(out_path))
    print(f'  Saved: {out_path} ({out_path.stat().st_size / 1e6:.1f} MB)')

## 6. Validation

In [9]:
# Reload and verify
for region in REGIONS:
    path = OUTPUT_DIR / f'Allen2022_surface.{region}.nc'
    da = xr.open_dataarray(str(path))
    da.load()
    
    # Check NC values
    nc_vals = da.coords['nc_testset'].values
    subj1_mask = da.coords['subject'].values == 'subj01'
    nc_subj1 = nc_vals[subj1_mask]
    
    # Filter to complete images
    complete_mask = da.coords['min_reps_across_subjects'].values >= 3
    da_complete = da.isel(presentation=complete_mask)
    
    print(f'{region}:')
    print(f'  Full: {da.shape}, Complete: {da_complete.shape}')
    print(f'  Subj01 median NC: {np.nanmedian(nc_subj1):.1f}%')
    print(f'  NaN in complete subset: {np.isnan(da_complete.values).sum()}')
    print()

V1:
  Full: (1000, 34208), Complete: (515, 34208)
  Subj01 median NC: 47.8%
  NaN in complete subset: 986

V2:
  Full: (1000, 27128), Complete: (515, 27128)
  Subj01 median NC: 49.1%
  NaN in complete subset: 343

V4:
  Full: (1000, 7312), Complete: (515, 7312)
  Subj01 median NC: 47.0%
  NaN in complete subset: 0

IT:
  Full: (1000, 89344), Complete: (515, 89344)
  Subj01 median NC: 22.7%
  NaN in complete subset: 0



In [10]:
print('Surface data preparation complete.')
print(f'Assemblies saved to: {OUTPUT_DIR}')
print(f'\nFiles:')
for region in REGIONS:
    path = OUTPUT_DIR / f'Allen2022_surface.{region}.nc'
    print(f'  {path.name}: {path.stat().st_size / 1e6:.1f} MB')

Surface data preparation complete.
Assemblies saved to: /Volumes/Hagibis/nsd/assemblies

Files:
  Allen2022_surface.V1.nc: 145.7 MB
  Allen2022_surface.V2.nc: 115.6 MB
  Allen2022_surface.V4.nc: 31.2 MB
  Allen2022_surface.IT.nc: 380.3 MB
